In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.formula.api import ols
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import seaborn as sns
sns.set_theme()
import pickle
from datetime import datetime
import pytz
from pytz import common_timezones
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('/Users/Mike/Flatiron_020121/king_county_phase2_project/king_county_property_values_analysis_predictive_model/data/kc_house_data_test_features.csv', index_col='Unnamed: 0')
print(df.shape)
df.head()

(4322, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [8]:
len(df)

4322

In [7]:
df.isna().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [5]:
df.dtypes

id                 int64
date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [9]:
# CLEANING: formatting date column from ICS datetime to datetime64(ns) for easier manipulation with Pandas
df.date = pd.to_datetime(df['date'], format='%Y/%m/%d') 

# creates variable representing today with format matching the cleaned 'date' column formatting
today = pd.to_datetime(pd.Timestamp.today(), format='%Y/%m/%d') 

# FE: number representing how many years ago (from today) that the sale occurred
df['yrs_since_sale'] = pd.to_numeric((today-df.date)/(3.154*10**16)) 

# FE: number representing age of property in years
df['yrs_old'] = 2021-df.yr_built 

# FE: how many years since renovation
df['yrs_since_renovation'] = 2021-df.yr_renovated 

# CLEANING: corrects properties that have never been renovated for engineered feature 'yrs_since_renovation'
df['yrs_since_renovation'] = np.where(df.yrs_since_renovation == 2021, 
                                      df.yrs_old, 
                                      df.yrs_since_renovation)

# FE: how many years since renovation at year of sale, create from 'yrs_since_renovation' minus 'yrs_since_sale'
df['yrs_since_ren_at_sale'] = (df.yrs_since_renovation - df.yrs_since_sale) 

# FE: subtracts total property livable sq. ft. divided by number of floors from lot size to estimate yard size 
df['sqft_yard'] = (df.sqft_lot-(df.sqft_living/df.floors)) 

# CLEANING: sets negative yard values to 0
df['sqft_yard'] = np.where(df.sqft_yard < 0, 
                                      0, 
                                      df.sqft_yard)

# CLEANING: sets 33 bedrooms to 3 (median and likely intended entry)
df['bedrooms'] = np.where(df.bedrooms == 33, 
                                      3, 
                                      df.bedrooms)

# CLEANING: caps total number of bedrooms to IQR max
df['bedrooms'] =  np.where(df['bedrooms'] >8, 
                                      8, 
                                      df.bedrooms)

# CLEANING: drops properties with 0 bedrooms
df['bedrooms'] =  np.where(df['bedrooms'] < 1, 
                                      1, 
                                      df.bedrooms)

# CLEANING: caps total number of bathrooms where number of bathrooms > 4 to IQR max
df['bathrooms'] =  np.where(df.bathrooms >4, 
                                      4, 
                                      df.bathrooms)

# CLEANING: caps properties total number of bathrooms where number of bathrooms < .75 to .75 bathrooms
df['bathrooms'] =  np.where(df.bathrooms <.75, 
                                      .75, 
                                      df.bathrooms)

# FE: creates a dummy variable similar to 'waterfront', that marks a property does not have 0 listed for the sqft of their basement 
df['sqft_basement_not_zero'] = np.where(df.sqft_basement == 0,
                                       1,
                                       0)

# CLEANING: caps properties with grades < 5 to grade 5
df['grade'] = np.where(df.grade < 5,
                       5,
                       df.grade)

# CLEANING: caps properties with grades > 11 to 11
df['grade'] = np.where(df.grade > 11, 
                       11, 
                       df.grade)

# DUMMY: creates dummy variable for rural or not
df['rural'] = np.where(df.long > -121.961527, 
                                      1, 
                                      0)

# DUMMY: creates dummy variable for if property is within Seattle city limits or not
df['within_seattle_city_limits'] = np.where((df.long < -122.251569)
                                            & (df.long > -122.438230)
                                            & (df.lat < 47.734178)
                                            & (df.lat > 47.495479),
                                            1, 
                                            0)  

# FE: creates 'sale_month' from 'date' column's datetime64 type data
df['sale_month'] = df.date.dt.month # isolating sale month from 'date' column datetime64 object

# FE: creates 'sale_year' from 'date' column's datetime64 type data
df['sale_year'] = df.date.dt.year 

# DUMMY: dummy variable columns for each season
df['sale_season_winter'] = np.where(((df.sale_month == 1)
                                   | (df.sale_month == 2)
                                   | (df.sale_month == 3)),
                                   1,
                                   0)

df['sale_season_spring'] = np.where(((df.sale_month == 4)
                            | (df.sale_month == 5)
                            | (df.sale_month == 6)),
                             1,
                             0)

df['sale_season_summer'] = np.where(((df.sale_month == 7)
                                   | (df.sale_month == 8)
                                   | (df.sale_month == 9)),
                                   1,
                                   0)

df['sale_season_fall'] = np.where(((df.sale_month == 10)
                                 | (df.sale_month == 11)
                                 | (df.sale_month == 12)),
                                  1,
                                  0)

# CLEANING: not relevant to analysis nor predictive model
df.drop(columns='id', inplace = True) 

In [19]:
df = pd.get_dummies(df, columns=['bedrooms', 'bathrooms', 'grade', 'zipcode', 'floors', 'view', 'condition'], drop_first=True)

In [22]:
df.drop(['date', 'yr_built', 'yr_renovated', 'lat', 'long', 'sqft_lot'], axis=1, inplace=True)

In [23]:
# df.isna().mean()
len(df) # just making sure I didn't drop anything!

4322

In [12]:
infile = open('model.pickle','rb')
model = pickle.load(infile)
infile.close()

In [13]:
print(model.intercept_)
print(len(model.coef_))

-73852836.99496628
128


In [14]:
infile = open('features.pickle','rb')
other_info = pickle.load(infile)
infile.close()

In [15]:
other_info

['sqft_living',
 'waterfront',
 'sqft_above',
 'sqft_basement',
 'sqft_living15',
 'sqft_lot15',
 'yrs_since_sale',
 'yrs_old',
 'yrs_since_renovation',
 'yrs_since_ren_at_sale',
 'sqft_yard',
 'sqft_basement_not_zero',
 'rural',
 'within_seattle_city_limits',
 'sale_month',
 'sale_year',
 'sale_season_winter',
 'sale_season_spring',
 'sale_season_summer',
 'sale_season_fall',
 'bedrooms_2',
 'bedrooms_3',
 'bedrooms_4',
 'bedrooms_5',
 'bedrooms_6',
 'bedrooms_7',
 'bedrooms_8',
 'bathrooms_1.0',
 'bathrooms_1.25',
 'bathrooms_1.5',
 'bathrooms_1.75',
 'bathrooms_2.0',
 'bathrooms_2.25',
 'bathrooms_2.5',
 'bathrooms_2.75',
 'bathrooms_3.0',
 'bathrooms_3.25',
 'bathrooms_3.5',
 'bathrooms_3.75',
 'bathrooms_4.0',
 'grade_6',
 'grade_7',
 'grade_8',
 'grade_9',
 'grade_10',
 'grade_11',
 'zipcode_98002',
 'zipcode_98003',
 'zipcode_98004',
 'zipcode_98005',
 'zipcode_98006',
 'zipcode_98007',
 'zipcode_98008',
 'zipcode_98010',
 'zipcode_98011',
 'zipcode_98014',
 'zipcode_98019',
 'z

In [16]:
df.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yrs_since_sale,yrs_old,yrs_since_renovation,yrs_since_ren_at_sale,sqft_yard,sqft_basement_not_zero,rural,within_seattle_city_limits,sale_month,sale_year,sale_season_winter,sale_season_spring,sale_season_summer,sale_season_fall
0,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,6,54,54,48,9230.0,0,0,0,8,2014,0,0,1,0
1,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,6,54,54,48,9230.0,0,0,0,2,2015,1,0,0,0
2,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,6,16,16,10,1044.0,0,0,0,11,2014,0,0,0,1
3,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,6,45,45,39,14920.0,0,0,0,12,2014,0,0,0,1
4,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,6,16,16,10,6711.0,1,0,0,1,2015,1,0,0,0


In [24]:
final_columns = list(other_info)

In [25]:
final_columns # checking to confirm I didn't blow it yet

['sqft_living',
 'waterfront',
 'sqft_above',
 'sqft_basement',
 'sqft_living15',
 'sqft_lot15',
 'yrs_since_sale',
 'yrs_old',
 'yrs_since_renovation',
 'yrs_since_ren_at_sale',
 'sqft_yard',
 'sqft_basement_not_zero',
 'rural',
 'within_seattle_city_limits',
 'sale_month',
 'sale_year',
 'sale_season_winter',
 'sale_season_spring',
 'sale_season_summer',
 'sale_season_fall',
 'bedrooms_2',
 'bedrooms_3',
 'bedrooms_4',
 'bedrooms_5',
 'bedrooms_6',
 'bedrooms_7',
 'bedrooms_8',
 'bathrooms_1.0',
 'bathrooms_1.25',
 'bathrooms_1.5',
 'bathrooms_1.75',
 'bathrooms_2.0',
 'bathrooms_2.25',
 'bathrooms_2.5',
 'bathrooms_2.75',
 'bathrooms_3.0',
 'bathrooms_3.25',
 'bathrooms_3.5',
 'bathrooms_3.75',
 'bathrooms_4.0',
 'grade_6',
 'grade_7',
 'grade_8',
 'grade_9',
 'grade_10',
 'grade_11',
 'zipcode_98002',
 'zipcode_98003',
 'zipcode_98004',
 'zipcode_98005',
 'zipcode_98006',
 'zipcode_98007',
 'zipcode_98008',
 'zipcode_98010',
 'zipcode_98011',
 'zipcode_98014',
 'zipcode_98019',
 'z

In [26]:
final_answers = model.predict(df[final_columns]) # fingers crossed

In [30]:
predictions = pd.DataFrame(final_answers)

In [41]:
pd.set_option('display.max_rows', 4400)
predictions

,0
0,5.017765e+05
1,5.157462e+05
2,3.684508e+05
3,2.467563e+05
4,5.076261e+05
5,5.681047e+05
6,2.762054e+05
7,3.023807e+05
8,3.811259e+05
9,2.732893e+05


In [43]:
predictions.to_csv('/Users/Mike/Flatiron_020121/king_county_phase2_project/king_county_property_values_analysis_predictive_model/data/housing_preds_Mike_Flanagan.csv')